In [6]:
import orjson
import os
import pandas as pd
from tqdm import tqdm
import orjson
import numpy as np
import pickle

In [39]:
tier = 'diamond'
output_dir = "/home/piddle/hdd/matches"
target_dir = os.path.join(output_dir, tier)

# 전처리

In [45]:
all_column=['match_id', 'game_length_second', 'summoner_id', 'summoner_level', 'champion_id', 'team_key', 'position',
             'trinket_item', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0', 'spell_1',
             'champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
             'total_damage_taken', 'total_damage_dealt', 'total_damage_dealt_to_champions', 'time_ccing_others',
             'vision_score', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place',
             'turret_kill', 'kill', 'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal', 'result']
    
cols1 = ['champion_id', 'team_key', 'position', 'trinket_item']

stat_cols = ['champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
            'total_damage_taken', 'total_damage_dealt', 'total_damage_dealt_to_champions', 'time_ccing_others',
            'time_ccing_others', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place',
            'turret_kill', 'kill', 'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal']

match_col = ['match_id', 'team_key', 'summoner_id', 'result']

In [46]:
def parse_match(file_iter) -> pd.DataFrame:
    match_by_match_id_chunk = []
    match_by_summoner_id_chunk = []
    for n, file in enumerate(tqdm(file_iter)):
        with open(file.path, 'rb') as f:
            try:
                json_data = orjson.loads(f.read())
            except Exception as e:
                print('Error reading {}: {}'.format(file.path, e))
                continue

        summoner_id = file.name[:-5]

        if not 'participants' in json_data[0]:
            continue
        
        for match in json_data:
            for participant in match['participants']:
                
                if match['game_length_second'] == 0:
                    continue
                
                match_by_match_id = {}

                match_by_match_id['match_id'] = match['id']
                match_by_match_id['summoner_id'] = participant['summoner']['summoner_id']
                match_by_match_id['team_key'] = participant['team_key']
                match_by_match_id['result'] = participant['stats']['result']

                match_by_match_id_chunk.append(match_by_match_id)

                if participant['summoner']['summoner_id'] == summoner_id:
                    match_by_summoner_id = {}
                    match_by_summoner_id['match_id'] = match['id']
                    match_by_summoner_id['game_length_second'] = match['game_length_second']

                    match_by_summoner_id['summoner_id'] = participant['summoner']['summoner_id']
                    match_by_summoner_id['summoner_level'] = participant['summoner']['level']

                    for col in cols1:
                        match_by_summoner_id[col] = participant[col]

                    for i, item in enumerate(participant['items']):
                        match_by_summoner_id[f'item_{i}'] = item

                    match_by_summoner_id['rune_0'] = participant['rune']["primary_rune_id"]
                    match_by_summoner_id['rune_1'] = participant['rune']["secondary_page_id"]
                    match_by_summoner_id['spell_0'] = participant['spells'][0]
                    match_by_summoner_id['spell_1'] = participant['spells'][1]

                    stats = participant['stats']
                    for col in stat_cols:
                        match_by_summoner_id[col] = stats[col]

                    match_by_summoner_id['vision_score'] = stats['vision_score']
                    match_by_summoner_id['result'] = stats['result']

                    match_by_summoner_id_chunk.append(match_by_summoner_id)


        if n % 20000 == 0:
            match_df = pd.DataFrame(match_by_match_id_chunk, columns=match_col)
            summoner_df = pd.DataFrame(match_by_summoner_id_chunk, columns=all_column)
            
            match_by_match_id_chunk = []
            match_by_summoner_id_chunk = []
            yield match_df, summoner_df

    else:
        match_df = pd.DataFrame(match_by_match_id_chunk, columns=match_col)
        summoner_df = pd.DataFrame(match_by_summoner_id_chunk, columns=all_column)
        yield match_df, summoner_df

In [47]:
file_iter = os.scandir(target_dir)
pd.DataFrame(columns=match_col).to_csv(f'../data/{tier}_match_by_match.csv', mode='w', index=False)
pd.DataFrame(columns=all_column).to_csv(f'../data/{tier}_match_by_summoner.csv', mode='w', index=False)

for match_df, summoner_df in parse_match(file_iter):
    match_df.to_csv(f'../data/{tier}_match_by_match.csv', mode='a', index=False, header=False)
    summoner_df.to_csv(f'../data/{tier}_match_by_summoner.csv', mode='a', index=False, header=False)

82586it [25:58, 52.98it/s]


# SUMMONER

In [ ]:
target = os.listdir(target_dir)
for i, t in enumerate(target):
    target[i] = t[:-5]

In [ ]:
summoner_df = pd.read_csv(f'../data/{tier}_match_by_summoner.csv')

In [135]:
summoner_df = summoner_df.drop_duplicates(['match_id', 'summoner_id'])
summoner_df

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,1560,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,13,BLUE,MID,3363,4630,6657,...,0,9,1,1,4,3,12,8334,2751,LOSE
1,CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXkSD5CEs=,1320,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,BLUE,MID,3363,3100,3115,...,1,3,0,2,0,2,0,8892,2896,LOSE
2,CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92YA3Sjg=,1418,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,3113,3115,...,0,6,0,2,5,0,0,7683,57,LOSE
3,CNrwP4AK2Xc7lbRKnMKmbGRWJa5HoFEi1ukJgLKusn4=,1558,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,1058,2420,...,2,9,1,6,1,6,0,10677,1816,WIN
4,CNrwP4AK2XdtwndtdTggt5-DDndWH9WOTCTcDueS91o=,1740,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,3,BLUE,MID,3340,3152,2055,...,0,13,0,2,6,11,0,8925,2265,LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,CNrwP4AK2XcuLAUwGjRcPsYz7LHzsHWQx9rbXkhM8-o=,1375,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,1055,3091,...,1,7,3,8,4,3,13,13239,11841,WIN
824672,CNrwP4AK2XcJUumytI7EkP5v8HhShIWuoki3rnQOibQ=,2419,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,777,RED,MID,3364,3031,6665,...,6,7,1,15,9,5,38,20759,21222,WIN
824673,CNrwP4AK2XdMX2UtgEPveuUii4UNF1jpcsl8izOQo2U=,1001,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,134,RED,MID,3363,1056,3113,...,2,7,0,1,2,2,0,6358,401,WIN
824674,CNrwP4AK2XdJ2fmOdyFbYc5dz4Ae6E7FoDzlksEGre4=,1681,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,6610,3123,...,2,9,1,7,5,1,4,14105,13533,LOSE


In [136]:
summoner_df = summoner_df[summoner_df['result'].isin(['WIN', 'LOSE'])]
summoner_df

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,1560,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,13,BLUE,MID,3363,4630,6657,...,0,9,1,1,4,3,12,8334,2751,LOSE
1,CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXkSD5CEs=,1320,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,BLUE,MID,3363,3100,3115,...,1,3,0,2,0,2,0,8892,2896,LOSE
2,CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92YA3Sjg=,1418,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,3113,3115,...,0,6,0,2,5,0,0,7683,57,LOSE
3,CNrwP4AK2Xc7lbRKnMKmbGRWJa5HoFEi1ukJgLKusn4=,1558,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,268,RED,MID,3363,1058,2420,...,2,9,1,6,1,6,0,10677,1816,WIN
4,CNrwP4AK2XdtwndtdTggt5-DDndWH9WOTCTcDueS91o=,1740,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,266,3,BLUE,MID,3340,3152,2055,...,0,13,0,2,6,11,0,8925,2265,LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,CNrwP4AK2XcuLAUwGjRcPsYz7LHzsHWQx9rbXkhM8-o=,1375,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,1055,3091,...,1,7,3,8,4,3,13,13239,11841,WIN
824672,CNrwP4AK2XcJUumytI7EkP5v8HhShIWuoki3rnQOibQ=,2419,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,777,RED,MID,3364,3031,6665,...,6,7,1,15,9,5,38,20759,21222,WIN
824673,CNrwP4AK2XdMX2UtgEPveuUii4UNF1jpcsl8izOQo2U=,1001,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,134,RED,MID,3363,1056,3113,...,2,7,0,1,2,2,0,6358,401,WIN
824674,CNrwP4AK2XdJ2fmOdyFbYc5dz4Ae6E7FoDzlksEGre4=,1681,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,83,39,BLUE,TOP,3364,6610,3123,...,2,9,1,7,5,1,4,14105,13533,LOSE


In [137]:
summoner_df.columns

Index(['match_id', 'game_length_second', 'summoner_id', 'summoner_level',
       'champion_id', 'team_key', 'position', 'trinket_item', 'item_0',
       'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0',
       'spell_1', 'champion_level', 'damage_self_mitigated',
       'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
       'total_damage_taken', 'total_damage_dealt',
       'total_damage_dealt_to_champions', 'time_ccing_others', 'vision_score',
       'vision_wards_bought_in_game', 'sight_wards_bought_in_game',
       'ward_kill', 'ward_place', 'turret_kill', 'kill', 'death', 'assist',
       'neutral_minion_kill', 'gold_earned', 'total_heal', 'result'],
      dtype='object')

In [138]:
cate_col = ['match_id', 'summoner_id', 'champion_id', 'team_key', 'position', 'trinket_item', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0', 'spell_1', 'result']
cont_col = ['game_length_second', 'summoner_level', 'champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets', 'total_damage_taken', 'total_damage_dealt', 
            'total_damage_dealt_to_champions', 'time_ccing_others', 'vision_score', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place', 'turret_kill', 'kill', 
            'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal']

In [139]:
for col in cate_col:
    summoner_df[col] = summoner_df[col].apply(lambda x: col + '.' + str(x))
summoner_df

/tmp/ipykernel_3352/1842175635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summoner_df[col] = summoner_df[col].apply(lambda x: col + '.' + str(x))


,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,1560,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.13,team_key.BLUE,position.MID,trinket_item.3363,item_0.4630,item_1.6657,...,0,9,1,1,4,3,12,8334,2751,result.LOSE
1,match_id.CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXk...,1320,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.268,team_key.BLUE,position.MID,trinket_item.3363,item_0.3100,item_1.3115,...,1,3,0,2,0,2,0,8892,2896,result.LOSE
2,match_id.CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92...,1418,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.268,team_key.RED,position.MID,trinket_item.3363,item_0.3113,item_1.3115,...,0,6,0,2,5,0,0,7683,57,result.LOSE
3,match_id.CNrwP4AK2Xc7lbRKnMKmbGRWJa5HoFEi1ukJg...,1558,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.268,team_key.RED,position.MID,trinket_item.3363,item_0.1058,item_1.2420,...,2,9,1,6,1,6,0,10677,1816,result.WIN
4,match_id.CNrwP4AK2XdtwndtdTggt5-DDndWH9WOTCTcD...,1740,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,266,champion_id.3,team_key.BLUE,position.MID,trinket_item.3340,item_0.3152,item_1.2055,...,0,13,0,2,6,11,0,8925,2265,result.LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,match_id.CNrwP4AK2XcuLAUwGjRcPsYz7LHzsHWQx9rbX...,1375,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.39,team_key.BLUE,position.TOP,trinket_item.3364,item_0.1055,item_1.3091,...,1,7,3,8,4,3,13,13239,11841,result.WIN
824672,match_id.CNrwP4AK2XcJUumytI7EkP5v8HhShIWuoki3r...,2419,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.777,team_key.RED,position.MID,trinket_item.3364,item_0.3031,item_1.6665,...,6,7,1,15,9,5,38,20759,21222,result.WIN
824673,match_id.CNrwP4AK2XdMX2UtgEPveuUii4UNF1jpcsl8i...,1001,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.134,team_key.RED,position.MID,trinket_item.3363,item_0.1056,item_1.3113,...,2,7,0,1,2,2,0,6358,401,result.WIN
824674,match_id.CNrwP4AK2XdJ2fmOdyFbYc5dz4Ae6E7FoDzlk...,1681,summoner_id.ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203O...,83,champion_id.39,team_key.BLUE,position.TOP,trinket_item.3364,item_0.6610,item_1.3123,...,2,9,1,7,5,1,4,14105,13533,result.LOSE


In [ ]:
cate = {}
cate['match_id'] = ['match_id']
cate['summoner_id'] = ['summoner_id']
cate['other'] = ['champion_id', 'team_key', 'position', 'trinket_item', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0', 'spell_1', 'result']

In [140]:
unique = {}
for key in cate.keys():
    unique[key] = np.array([])
    for col in cate[key]:
        unique[key] = np.concatenate((unique[key], summoner_df[col].unique()))
 
unique

array(['match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=',
       'match_id.CNrwP4AK2Xe1SGRd-otNS5nV6TMvh7jgs2tXkSD5CEs=',
       'match_id.CNrwP4AK2Xdi2_kFJSvTtn2dBruz3jlZt1W92YA3Sjg=', ...,
       'spell_1.3', 'result.LOSE', 'result.WIN'], dtype=object)

In [141]:
to_index = {}

for key in cate.keys():
    to_index[key] = {}
    ### mask index 0을 생성.
    for i, u in enumerate(unique[key], start=1):
        to_index[key][u] = i

with open('../data/to_index.pkl', 'wb') as file:
    pickle.dump(to_index, file)

In [142]:
for key in cate.keys():
    for col in cate[key]:
        summoner_df[col] = summoner_df[col].map(to_index[key])

summoner_df

/tmp/ipykernel_3352/219301454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summoner_df[col] = summoner_df[col].map(to_index)
/tmp/ipykernel_3352/219301454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summoner_df[col] = summoner_df[col].map(to_index)
/tmp/ipykernel_3352/219301454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,1,1560,477959,266,560532,560699,560701,560707,560713,560981,...,0,9,1,1,4,3,12,8334,2751,562068
1,2,1320,477959,266,560533,560699,560701,560707,560714,560982,...,1,3,0,2,0,2,0,8892,2896,562068
2,3,1418,477959,266,560533,560700,560701,560707,560715,560982,...,0,6,0,2,5,0,0,7683,57,562068
3,4,1558,477959,266,560533,560700,560701,560707,560716,560983,...,2,9,1,6,1,6,0,10677,1816,562069
4,5,1740,477959,266,560534,560699,560701,560708,560717,560984,...,0,13,0,2,6,11,0,8925,2265,562068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824671,461376,1375,560531,83,560545,560699,560703,560709,560725,561007,...,1,7,3,8,4,3,13,13239,11841,562069
824672,119956,2419,560531,83,560557,560700,560701,560709,560738,561061,...,6,7,1,15,9,5,38,20759,21222,562069
824673,402337,1001,560531,83,560649,560700,560701,560707,560720,561089,...,2,7,0,1,2,2,0,6358,401,562069
824674,41765,1681,560531,83,560545,560699,560703,560709,560792,561082,...,2,9,1,7,5,1,4,14105,13533,562068


In [143]:
summoner_df = summoner_df.astype(int)
summoner_df.to_csv(f'../data/{tier}_match_by_summoner_mod.csv', mode='w', index=False)

# Match

In [41]:
match_df = pd.read_csv(f'../data/{tier}_match_by_match.csv')
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,xe4NOfPaQy8aUvWRPjo-9J_daUe1poo-yEWgFn6aCj2vRA,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
...,...,...,...,...
8246755,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,HGGccfhZJmTDsGjcJlwkUhpjdowQ2pKm74cUNtmmO52sgCE,LOSE
8246756,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,52cRsBkYTek54NCxFRe5RiotLYsAnNGbJrGQLJ3VNv146hA,WIN
8246757,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE
8246758,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,LOSE


In [42]:
match_df = match_df.drop_duplicates(['match_id', 'summoner_id'])
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,xe4NOfPaQy8aUvWRPjo-9J_daUe1poo-yEWgFn6aCj2vRA,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
...,...,...,...,...
8246755,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,HGGccfhZJmTDsGjcJlwkUhpjdowQ2pKm74cUNtmmO52sgCE,LOSE
8246756,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,52cRsBkYTek54NCxFRe5RiotLYsAnNGbJrGQLJ3VNv146hA,WIN
8246757,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE
8246758,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,ULAwYO4uGJy-JsnKTRGhPsV5pgUw4i203OxBVRVyUcjm0O...,LOSE


In [43]:
target_bool = match_df['summoner_id'].isin(target)
match_df = match_df[target_bool].reset_index(drop=True)
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,O-iLgR6I7C58cMpFzyduQlJxZevdfoknb2-4bk1fp4I9dNc,WIN
...,...,...,...,...
3068073,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,MjiYweessMwizZmID6fl7HtJ2l52-kjaCHKw3BpJD4YXqw,WIN
3068074,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,KMxsTXOsx3b0g0ZndKZF3ERjnn4BlV5O56LovLidW4HIZaI,WIN
3068075,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,oPdQVPS1xVpz5WGTQqH3go_qyMoSBGNRyOkEDwPsL6qtZnE,LOSE
3068076,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE


In [44]:
match_df = match_df[match_df['result'].isin(['WIN', 'LOSE'])]
match_df

,match_id,team_key,summoner_id,result
0,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r-fHgcIGknzMw,LOSE
1,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,BLUE,Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmjpP2IiQPRn-xz4,LOSE
2,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmpP0lXG2MJ7G1pQ,WIN
3,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,vgSDdCghIp2Yk0HLfDNrL8JzET6zfihaclpOLHrtDiw7yqU,WIN
4,CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth7rns60=,RED,O-iLgR6I7C58cMpFzyduQlJxZevdfoknb2-4bk1fp4I9dNc,WIN
...,...,...,...,...
3068073,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,MjiYweessMwizZmID6fl7HtJ2l52-kjaCHKw3BpJD4YXqw,WIN
3068074,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,BLUE,KMxsTXOsx3b0g0ZndKZF3ERjnn4BlV5O56LovLidW4HIZaI,WIN
3068075,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,oPdQVPS1xVpz5WGTQqH3go_qyMoSBGNRyOkEDwPsL6qtZnE,LOSE
3068076,CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnNF3aYyg=,RED,hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3AW3thpbDE6HcX...,LOSE


In [45]:
for col in ['match_id', 'team_key', 'summoner_id']:
    match_df[col] = match_df[col].apply(lambda x: col + '.' + str(x))

match_df

/tmp/ipykernel_22069/2471060713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[col] = match_df[col].apply(lambda x: col + '.' + str(x))


,match_id,team_key,summoner_id,result
0,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.BLUE,summoner_id.PBMwh49a37I5BGawSE6CD5nI_cQLoSOd1r...,LOSE
1,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.BLUE,summoner_id.Pki8T7HBL-aeecO7mmcMIFeKWWRc-zPfmj...,LOSE
2,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.RED,summoner_id.WaPHpyjd2fRm2eRoM6SKYXZyi6YYVYnKmp...,WIN
3,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.RED,summoner_id.vgSDdCghIp2Yk0HLfDNrL8JzET6zfihacl...,WIN
4,match_id.CNrwP4AK2Xd8w3fsBE51tSQaA30xjKBw8Axth...,team_key.RED,summoner_id.O-iLgR6I7C58cMpFzyduQlJxZevdfoknb2...,WIN
...,...,...,...,...
3068073,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.BLUE,summoner_id.MjiYweessMwizZmID6fl7HtJ2l52-kjaCH...,WIN
3068074,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.BLUE,summoner_id.KMxsTXOsx3b0g0ZndKZF3ERjnn4BlV5O56...,WIN
3068075,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.RED,summoner_id.oPdQVPS1xVpz5WGTQqH3go_qyMoSBGNRyO...,LOSE
3068076,match_id.CNrwP4AK2Xf4b_NkMm25QLQHEBZI16rXEHhnN...,team_key.RED,summoner_id.hFKtUw6JHXyLINBTwOi1Cd8m-Lp0LFU_3A...,LOSE


In [ ]:
cate = {}
cate['match_id'] = ['match_id']
cate['summoner_id'] = ['summoner_id']
cate['other'] = ['team_key', 'result']

In [46]:
with open('../data/to_index.pkl', 'rb') as file:
    to_index = pickle.load(file)

to_index['other']['LOSE'] = 0
to_index['other']['WIN'] = 1

for key in cate.keys():
    for col in cate[key]:
        match_df[col] = match_df[col].map(to_index[key])

match_df

/tmp/ipykernel_22069/426253025.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[col] = match_df[col].map(to_index)
/tmp/ipykernel_22069/426253025.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[col] = match_df[col].map(to_index)
/tmp/ipykernel_22069/426253025.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,match_id,team_key,summoner_id,result
0,1,560699,518555.0,0
1,1,560699,477959.0,0
2,1,560700,509625.0,1
3,1,560700,505123.0,1
4,1,560700,494939.0,1
...,...,...,...,...
3068073,477958,560699,486581.0,1
3068074,477958,560699,531954.0,1
3068075,477958,560700,538028.0,0
3068076,477958,560700,526480.0,0


In [47]:
nna = match_df['summoner_id'].notna()
match_df = match_df[nna]
match_df

,match_id,team_key,summoner_id,result
0,1,560699,518555.0,0
1,1,560699,477959.0,0
2,1,560700,509625.0,1
3,1,560700,505123.0,1
4,1,560700,494939.0,1
...,...,...,...,...
3068073,477958,560699,486581.0,1
3068074,477958,560699,531954.0,1
3068075,477958,560700,538028.0,0
3068076,477958,560700,526480.0,0


In [48]:
temp = match_df.groupby(['match_id', 'team_key']).size()
valid_indices = temp[temp > 1].index
match_df = match_df[match_df.set_index(['match_id', 'team_key']).index.isin(valid_indices)]
match_df

,match_id,team_key,summoner_id,result
0,1,560699,518555.0,0
1,1,560699,477959.0,0
2,1,560700,509625.0,1
3,1,560700,505123.0,1
4,1,560700,494939.0,1
...,...,...,...,...
3068073,477958,560699,486581.0,1
3068074,477958,560699,531954.0,1
3068075,477958,560700,538028.0,0
3068076,477958,560700,526480.0,0


In [49]:
match_df = match_df.astype(int)
match_df.to_csv(f'../data/{tier}_match_by_match_mod.csv', mode='w', index=False)

# DATASET

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F
from torch import nn
from sklearn.preprocessing import MinMaxScaler

import itertools
import math

In [50]:
tier = 'diamond'
match_df = pd.read_csv(f'../data/{tier}_match_by_match_mod.csv')
summoner_df = pd.read_csv(f'../data/{tier}_match_by_summoner_mod.csv')

In [52]:
cate_col = ['champion_id', 'team_key', 'position', 'trinket_item', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0', 'spell_1', 'result']
cont_col = ['game_length_second', 'summoner_level', 'champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets', 'total_damage_taken', 'total_damage_dealt', 
            'total_damage_dealt_to_champions', 'time_ccing_others', 'vision_score', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place', 'turret_kill', 'kill', 
            'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal']

In [53]:
class CFG:
    def __init__(self, config_file: str = None):
        self._store = {}
        if config_file is not None:
            extension = config_file.split(sep='.')[-1]
            if extension == 'json':
                import json
                form = json
                with open(config_file, 'r') as f:
                    config = form.load(f)

            elif extension == 'yaml':
                import yaml
                form = yaml
                with open(config_file, 'r') as f:
                    config = form.load(f, Loader=yaml.FullLoader)

            else:
                raise TypeError

            for key, value in config.items():
                self._store[key] = value
    
    def __getitem__(self, key):
        return self._store[key]
    
    def __setitem__(self, key, value):
        self._store[key] = value

    def __repr__(self):
        return str(self._store)

In [100]:
cfg = CFG()
cfg['cate_col'], cfg['cont_col'] = cate_col, cont_col
cfg['batch_size'] = 8
cfg['max_seq_len'] = 10
cfg['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

In [116]:
class MyDataset(Dataset):
    def __init__(self, cfg, summoner_df, match_df):
        self.cate_col = cfg['cate_col']
        self.cont_col = cfg['cont_col']
        self.max_seq_len = cfg['max_seq_len']    ### 10
        self.device = cfg['device']
        
        self.summoner_df, self.summoner_compression_table = self.prepare_summoner(summoner_df)
        self.match_df, self.len, self.combinations, self.match_compression_table = self.prepare_match(match_df)


    def prepare_summoner(self, df):
        df = df.sort_values(by='summoner_id')
        summoner_compression_table = []

        ### summoner_compression_table
        start = 0
        for end in df['summoner_id'].value_counts().sort_index().values:
            summoner_compression_table.append((start, start+end))
            start += end

        summoner_compression_table = np.array(summoner_compression_table)

        ### 게임 시간을 기준으로 나누어줌
        cont_col_mod = [item for item in self.cont_col if item != "game_length_second"]
        for col in cont_col_mod:
            df[col] /= df['game_length_second']

        ### 정규화
        scaler = MinMaxScaler()
        df[self.cont_col] = pd.DataFrame(scaler.fit_transform(df[self.cont_col]))

        return df, summoner_compression_table
    

    def prepare_match(self, df, k=2):
        df = df.sort_values(by='match_id')
        temp = df.groupby(['match_id', 'team_key']).size()

        len = temp.__len__()

        combinations = [[list(element) for element in itertools.combinations(range(count), 2)] for count in temp.values]

        match_compression_table = []
        start = 0
        for end in temp.values:
            match_compression_table.append((start, start+end))
            start += end

        match_compression_table = np.array(match_compression_table)
        
        return df, len, combinations, match_compression_table


    '''
    idx를 받으면 먼저 match 데이터에서 해당하는 match_id와 team_key에 맞는 데이터를 가져온다.
    해당하는 매치 데이터는 아군 최대 5개 최소 2개의 summoner_id와 1개의 동일한 result를 가지고 있다.
    매치데이터에서 summoner_id 2개를 뽑는 경우의 수 A, B를 구하고, 각 경우의 수에 맞게 summoner 데이터에서 해당하는 데이터를 가져온다.
    가져온 데이터는 cate, cont로 나누어져 있으며, cate는 embedding을 위해 int로, cont는 float로 변환한다.
    변환이 끝난 데이터는 A, B로 나누어 출력한다.
    '''
    def __getitem__(self, idx):
        ### 1개의 매치에 1개의 블루 or 레드팀의 5~2명의 팀원이 존재
        ### 팀원들 존재하는 iloc 위치를 가져옴
        match_start, match_end = self.match_compression_table[idx]
        match_df = self.match_df.iloc[match_start: match_end]

        summoner_compression_idx = match_df['summoner_id'].values - 1
        summoner_idx = self.summoner_compression_table[summoner_compression_idx]

        summoner_table_cate = []
        summoner_table_cont = []
        for summoner_start, summoner_end in summoner_idx:
            summoner_table_cate.append(self.summoner_df[summoner_start:summoner_end][self.cate_col].values)
            summoner_table_cont.append(self.summoner_df[summoner_start:summoner_end][self.cont_col].values)

        A_cate_data, A_cont_data, B_cate_data, B_cont_data = [], [], [], []
        combination = self.combinations[idx]
        for a, b in combination:
            A_cate = torch.zeros(self.max_seq_len, len(self.cate_col), dtype=torch.int)
            A_cate[:len(summoner_table_cate[a])] = torch.tensor(summoner_table_cate[a], dtype=torch.int)
            A_cate_data.append(A_cate)

            A_cont = torch.zeros(self.max_seq_len, len(self.cont_col), dtype=torch.float)
            A_cont[:len(summoner_table_cont[a])] = torch.tensor(summoner_table_cont[a], dtype=torch.float)
            A_cont_data.append(A_cont)

            B_cate = torch.zeros(self.max_seq_len, len(self.cate_col), dtype=torch.int)
            B_cate[:len(summoner_table_cate[b])] = torch.tensor(summoner_table_cate[b], dtype=torch.int)
            B_cate_data.append(B_cate)

            B_cont = torch.zeros(self.max_seq_len, len(self.cont_col), dtype=torch.float)
            B_cont[:len(summoner_table_cont[b])] = torch.tensor(summoner_table_cont[b], dtype=torch.float)
            B_cont_data.append(B_cont)
        
        axis0 = len(combination)
        A_cate_data = torch.stack(A_cate_data).view(axis0, -1)
        A_cont_data = torch.stack(A_cont_data).view(axis0, -1)
        B_cate_data = torch.stack(B_cate_data).view(axis0, -1)
        B_cont_data = torch.stack(B_cont_data).view(axis0, -1)
        
        result = match_df['result'].values[0]
        result = torch.tensor(result, dtype=torch.float).expand(axis0)

        return A_cate_data.to(self.device), A_cont_data.to(self.device), B_cate_data.to(self.device), B_cont_data.to(self.device), result.to(self.device)
        

    def __len__(self):
        return len(self.match_compression_table)
    


dataset = MyDataset(cfg, summoner_df, match_df)

In [117]:
dataset[11]

(tensor([[230655, 507870, 560557, 560699, 560701, 560708, 560809, 561006, 561331,
          561559, 561825, 562034, 562048, 562050, 562059, 562069, 230654, 507870,
          560539, 560699, 560701, 560708, 560732, 561000, 561254, 561515, 561776,
          562031, 562048, 562050, 562059, 562068, 230653, 507870, 560539, 560699,
          560701, 560709, 560723, 561048, 561257, 561528, 561795, 562031, 562048,
          562050, 562059, 562068, 230652, 507870, 560539, 560700, 560701, 560709,
          560730, 560998, 561275, 561530, 561795, 562031, 562048, 562050, 562059,
          562069, 230651, 507870, 560649, 560700, 560701, 560707, 560905, 561064,
          561296, 561514, 561776, 562039, 562044, 562050, 562059, 562068,      6,
          507870, 560539, 560699, 560701, 560709, 560782, 561008, 561246, 561528,
          561795, 562031, 562048, 562050, 562059, 562068, 230649, 507870, 560616,
          560700, 560704, 560709, 560754, 561063, 561301, 561655, 561771, 562038,
          562046

# DataLoader

In [118]:
def custom_collate_fn(batch):
    A, B, C, D, E = zip(*batch)

    A = torch.cat(A, dim=0)
    B = torch.cat(B, dim=0)
    C = torch.cat(C, dim=0)
    D = torch.cat(D, dim=0)
    E = torch.cat(E, dim=0)

    return A, B, C, D, E

In [119]:
dataset_size = len(dataset)
indices = list(range(dataset_size))

train_size = int(dataset_size * 0.8)
valid_size = dataset_size - train_size

torch.manual_seed(0)
# 인덱스를 무작위로 섞음 (데이터셋이 무작위로 분할되도록)
indices = torch.randperm(dataset_size).tolist()

train_indices = indices[:train_size]
valid_indices = indices[train_size:]

train_dataset = Subset(dataset, train_indices)
valid_dataset = Subset(dataset, valid_indices)

train_loader = DataLoader(train_dataset, batch_size=cfg['batch_size'], shuffle=True, collate_fn=custom_collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=cfg['batch_size'], shuffle=True, collate_fn=custom_collate_fn)

In [120]:
cfg['n_layers'] = summoner_df[cfg['cate_col']].max().max() + 1
cfg['emb_size'] = 3
cfg['hidden_size'] = 8

In [121]:
for a, b, c, d ,e in train_loader:
    print(a.size(), b.size(), c.size(), d.size(), e.size())
    break

torch.Size([43, 160]) torch.Size([43, 220]) torch.Size([43, 160]) torch.Size([43, 220]) torch.Size([43])


# Model

In [122]:
class SimilarityModel(nn.Module):
    def __init__(self, cfg):
        super(SimilarityModel, self).__init__()
        
        # categorical
        self.cate_emb = nn.Embedding(cfg['n_layers'], cfg['emb_size'], padding_idx=0)
        self.cate_proj = nn.Sequential(
            nn.Linear(cfg['max_seq_len'] * cfg['emb_size'] * len(cfg['cate_col']), cfg['hidden_size']),
            nn.LayerNorm(cfg['hidden_size']),
        )

        # continuous
        self.cont_proj = nn.Sequential(
            nn.BatchNorm1d(cfg['max_seq_len'] * len(cfg['cont_col'])),
            nn.Linear(cfg['max_seq_len'] * len(cfg['cont_col']), cfg['hidden_size']),
            nn.LayerNorm(cfg['hidden_size']),
        )

        # combination
        self.comb_proj = nn.Sequential(
            nn.ReLU(),
            nn.Linear(cfg['hidden_size'] * 2, cfg['hidden_size']),
            nn.LayerNorm(cfg['hidden_size'])
        )


    def forward(self, cate, cont):
        size = [i for i in cate.size()]
        cate = self.cate_emb(cate).view(*size[:-1], -1)
        cate = self.cate_proj(cate)
        
        cont = self.cont_proj(cont)

        output = self.comb_proj(torch.cat([cate, cont], dim=1))

        return output

In [123]:
model = SimilarityModel(cfg).to('cuda')

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

In [127]:
import joblib
from datetime import datetime
from sklearn.metrics import accuracy_score, roc_auc_score


def get_logger(logger_conf: dict):
    import logging
    import logging.config

    logging.config.dictConfig(logger_conf)
    logger = logging.getLogger()
    return logger


logging_conf = {  # only used when 'user_wandb==False'
    "version": 1,
    "formatters": {
        "basic": {"format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"}
    },
    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "INFO",
            "formatter": "basic",
            "stream": "ext://sys.stdout",
        },
        "file_handler": {
            "class": "logging.FileHandler",
            "level": "DEBUG",
            "formatter": "basic",
            "filename": "run.log",
        },
    },
    "root": {"level": "INFO", "handlers": ["console", "file_handler"]},
}

logger = get_logger(logger_conf=logging_conf)

In [125]:
model.train()
total_loss = 0
target_list = []
output_list = []
for A_cate, A_cont, B_cate, B_cont, result in tqdm(train_loader):
    optimizer.zero_grad()
    output1 = model(A_cate, A_cont)
    output2 = model(B_cate, B_cont)

    output = F.cosine_similarity(output1, output2, dim=1)
    
    
    loss = loss_fun(output, result) 
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    target_list.append(result.detach().cpu())
    output_list.append(output.detach().cpu())

acc = accuracy_score(y_true=target_list, y_pred=output_list > 0.5)
auc = roc_auc_score(y_true=target_list, y_score=output_list)
average_loss = total_loss / len(train_loader)

logger.info("TRAIN AUC : %.4f, ACC : %.4f, LOSS : %.4f", auc, acc, average_loss)

100%|██████████| 79555/79555 [54:14<00:00, 24.45it/s]  


In [ ]:
joblib.dump(model, f'../data/cossim_{datetime.now().strftime("%Y_%m_%d_%H_%M")}.joblib')

In [128]:
model.eval()
total_loss = 0
target_list = []
output_list = []
with torch.no_grad():
    for A_cate, A_cont, B_cate, B_cont, result in tqdm(valid_loader):
        output1 = model(A_cate, A_cont)
        output2 = model(B_cate, B_cont)

        output = F.cosine_similarity(output1, output2, dim=1)

        loss = loss_fun(output, result)
        total_loss += loss.item()
        target_list.append(result.detach().cpu())
        output_list.append(output.detach().cpu())

    target_list = torch.concat(target_list).numpy()
    output_list = torch.concat(output_list).numpy()

    acc = accuracy_score(y_true=target_list, y_pred=output_list > 0.5)
    auc = roc_auc_score(y_true=target_list, y_score=output_list)
    average_loss = total_loss / len(train_loader)

    logger.info("VALID AUC : %.4f, ACC : %.4f, LOSS : %.4f", auc, acc, average_loss)

100%|██████████| 19889/19889 [12:51<00:00, 25.79it/s]


TypeError: '>' not supported between instances of 'list' and 'float'

In [129]:
target_list = torch.concat(target_list).numpy()
output_list = torch.concat(output_list).numpy()

acc = accuracy_score(y_true=target_list, y_pred=output_list > 0.5)
auc = roc_auc_score(y_true=target_list, y_score=output_list)
average_loss = total_loss / len(train_loader)

logger.info("VALID AUC : %.4f, ACC : %.4f, LOSS : %.4f", auc, acc, average_loss)

2024-03-12 19:41:16,632 - root - INFO - VALID AUC : 0.5038, ACC : 0.5039, LOSS : 21.0478
